In [ ]:
!pip install kaggle pyspark textblob nltk


In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mv "kaggle (1).json" "kaggle.json"
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets list


In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews


In [ ]:
!unzip amazon-fine-food-reviews.zip


In [ ]:
!rm "kaggle (1).json"


In [ ]:
# Déplacer le fichier kaggle.json dans le répertoire attendu
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json




In [ ]:
!kaggle datasets list


In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews


In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews --force


In [ ]:
!unzip -o amazon-fine-food-reviews.zip


In [ ]:
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from textblob import TextBlob
import matplotlib.pyplot as plt

# Initialiser Spark
spark = SparkSession.builder.appName("Analyse de Sentiments").getOrCreate()

# Charger les données
df = spark.read.csv("Reviews.csv", header=True, inferSchema=True)

# Fonction de nettoyage du texte
def preprocess_text(text):
    if text is None:
        return ""
    # Retirer les caractères spéciaux et les URL
    text = re.sub(r'[^\w\s]', '', text)
    return text

# UDF pour nettoyer le texte
preprocess_udf = udf(preprocess_text, StringType())

# Appliquer le nettoyage
df = df.withColumn("cleaned_text", preprocess_udf(df["Text"]))

# Fonction d'analyse de sentiment
def get_sentiment(text):
    if text is None or text == "":
        return "neutre"
    # Analyser le sentiment avec TextBlob
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0:
        return "positif"
    elif polarity < 0:
        return "négatif"
    else:
        return "neutre"

# UDF pour l'analyse de sentiment
sentiment_udf = udf(get_sentiment, StringType())

# Appliquer l'analyse de sentiment
df = df.withColumn("sentiment", sentiment_udf(df["cleaned_text"]))

# Comptabiliser le nombre de chaque sentiment
sentiment_counts = df.groupBy("sentiment").count().collect()

# Afficher les résultats
sentiments = [row['sentiment'] for row in sentiment_counts]
counts = [row['count'] for row in sentiment_counts]

# Tracer un graphique
plt.bar(sentiments, counts, color=['blue', 'orange', 'green'])
plt.xlabel('Sentiment')
plt.ylabel('Nombre de Critiques')
plt.title('Répartition des Sentiments des Critiques de Produits')
plt.show()

# Arrêter Spark
spark.stop()
